In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from scipy import stats

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View all of the classes (tables) that automap found 
Base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table in hawaii.sqlite
MS = Base.classes.measurement
ST = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# View database
session.query(MS, ST).all()

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
# desc is descending and first is the first response
session.query(MS.date).order_by(MS.date.desc()).first()

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
date = dt.date(2017,8,23)- dt.timedelta(days=365)
print("date :", date)

# Perform a query to retrieve the data and precipitation scores
annual_rain = session.query(MS.date, MS.prcp).filter(MS.date >= date).all()
annual_rain

# Save the query results as a Pandas DataFrame and set the index to the date column
rain = pd.DataFrame(annual_rain, columns=['Date', 'Prcp'])
rain.set_index('Date', inplace=True)

# Sort the dataframe by date
rain.sort_index()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
rain.plot(rot = 50, title='Annual Precipitation')
plt.ylabel('Precipitation (in)')
plt.tight_layout
plt.savefig("Images/Annual_Precipitation.png")
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
rain.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_count = session.query(ST.station).group_by(ST.id).count()
print(f"The total number stations in the dataset is: {station_count}")

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)

# List the stations and the counts in descending order.
active =session.query(MS.station).group_by(MS.station).\
    order_by(func.count(MS.date).desc()).first()
most_active= active[0]
most_active

# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature
print(f"The station id of the most active station is {most_active}.")

In [ ]:
# Statistical analysis of the most active station
session.query(func.min(MS.tobs), func.max(MS.tobs), func.avg(MS.tobs)).\
filter(MS.station == most_active) .\
all()

print(f'''
--------------------------------------------------------------------
The lowest temperature recorded at the most active station is: 54.
The highest temperature recorded at the most active station is: 85.
The average temperature recorded at the most active station is: 72. 
---------------------------------------------------------------------
''')

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temp_active= session.query(MS.tobs).filter(MS.station=='USC00519281').\
    filter(MS.date>=date).all()
#temp_active.head() 

#Make a dataframe bc temp_active is a tuple ('list' object)
temp_df = pd.DataFrame(temp_active, columns=["tobs"])
#temp_df.head()

#temp_active
plt.hist(temp_df["tobs"], color ='green', bins=12)
plt.ylabel("Frequency")
plt.xlabel('Temperature (*F)')
plt.title('Most Active Rainfall Station Temperatures',
          fontweight ="bold")
plt.legend(["tobs"])
plt.tight_layout()
plt.show()

# Save the plot
plt.savefig("Images/Most_Active_Histogram.png")


# Close session

In [ ]:
# Close Session
#session.close()